In [49]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import networkx as nx
import sys
import math

In [114]:
df = pd.read_excel("pp.xlsx")

In [115]:
nodes = list(df['J1'].unique())
G = nx.MultiDiGraph()
G.add_nodes_from(nodes)
edges = list(zip(list(df['J1']),list(df['J2']),list(df['Quantity'])))

In [116]:
G.add_weighted_edges_from(edges)

In [117]:
iq_list = []
iq = 0
for i in nodes:
    for j in G.predecessors(i):
        edge_dict = G.get_edge_data(j,i)
        for k in range(0, G.number_of_edges(j,i)):
            iq = iq + edge_dict[k]['weight']
    iq_list.append(iq);
    iq = 0
for i in range(len(iq_list)):
    iq_list[i] = round(iq_list[i],3)

iq_dict = dict(zip(nodes,iq_list))
#iq_dict


In [118]:
br = int(input("Enter branch no..."))
changed_q = float(input("Enter new quantity..."))
a = df.iloc[br-1]['J1']
b = df.iloc[br-1]['J2']
q = df.iloc[br-1]['Quantity']
my_dict = G.get_edge_data(a,b)
for i in range(0,G.number_of_edges(a,b)):
    if(my_dict[i]['weight']==q):
        k=i
print(a,b)
my_dict[k]['weight'] = changed_q
print(G.get_edge_data(a,b))

Enter branch no...156
Enter new quantity...140
115 117
{0: {'weight': 140.0}}


In [119]:
#fwd_redistribution
def fwd_final_quant(t2):
    tq = 0
    for i in G.predecessors(t2):
        edge_dict = G.get_edge_data(i,t2)
        for j in range(0, G.number_of_edges(i,t2)):
            tq = tq + edge_dict[j]['weight']
    return tq

In [120]:
#fwd_redistribution
def fwd_redistribution(t1,t2):
    iq = iq_dict[t2]
    fq = fwd_final_quant(t2)
    for i in G.successors(t2):
        re_dict = G.get_edge_data(t2,i)
        for j in range(0, G.number_of_edges(t2,i)):
            re_dict[j]['weight'] = ((re_dict[j]['weight'])*(fq/iq))
            #if G.get_edge_data(12,1)[0]['weight']>sys.float_info.max:
                #G.get_edge_data(12,1)[0]['weight']=math.floor(sys.float_info.max)

In [121]:
#fwd_redistribution
t1 = a
t2 = b
destination = [b]
source = [a]
while(len(destination) != 0):
    t1 = source[0]
    t2 = destination[0]
    for i in G.successors(t2):
        if t2 in [9,12,39]:
            break
        destination.append(i)
        source.append(t2)
    
    fwd_redistribution(source.pop(0), destination.pop(0))
    print(t1,t2)
    #print(destination)

115 117
117 120
120 28
120 125
28 36
125 13
36 45
13 21
45 54
21 28
54 60
28 36
60 68
60 69
36 45
68 77
69 48
45 54
77 69
48 9
54 60
69 48
60 68
60 69
48 9
68 77
69 48
77 69
48 9
69 48
48 9


In [138]:
G.get_edge_data(109,114)[0]['weight']

1109.0729806635895

In [127]:
#backward distribution
t1 = a
t2 = b
destination = [b]
source = [a]
while(len(destination) != 0):
    t1 = source[0]
    t2 = destination[0]
    for i in G.predecessors(t1):
        if t1 in [1]:
            break
        destination.append(t1)
        source.append(i)
    
    back_redistribution(source.pop(0),destination.pop(0))
    print(t1,t2)
    #print(destination)

115 117
109 115
114 115
103 109
109 114
98 103
103 109
91 98
98 103
81 91
1 91
91 98
29 81
1 81
81 91
1 91
22 29
29 81
1 81
14 22
1 22
22 29
1 14
14 22
1 22
1 14


In [125]:
#back_redistribution
def back_final_quant(t1):
    tq = 0
    for i in G.successors(t1):
        edge_dict = G.get_edge_data(t1,i)
        for j in range(0, G.number_of_edges(t1,i)):
            tq = tq + edge_dict[j]['weight']
    return tq

In [126]:
#back_redistribution
def back_redistribution(t1,t2):
    iq = iq_dict[t1]
    fq = back_final_quant(t1)
    for i in G.predecessors(t1):
        re_dict = G.get_edge_data(i,t1)
        for j in range(0, G.number_of_edges(i,t1)):
            re_dict[j]['weight'] = ((re_dict[j]['weight'])*(fq/iq))